In [26]:
import pandas as pd
import pymorphy2
from collections import Counter
import csv
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
df = pd.read_csv('./data/covid_tweets.csv.gz', compression='gzip')

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# размеры данных
df.shape

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1491177, 3)

In [13]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text: str) -> list:
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Python38\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Python38\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Python38\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or ins

In [16]:
# получаем список предложений
lemmatized_data = []
for text in df['text'].head(500_000):
    lemmatized_data.append(lemmatize(text))

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# получаем список стоп-слов
with open('./data/stopwords-ru.txt', 'r', encoding='utf-8') as f:
    stopwords = [str(i) for i in f.read().split()]

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# удаляем стоп-слова если таковые есть
for sentence in lemmatized_data:
    for word in sentence:
        if word in stopwords:
            sentence.remove(word)

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
for sentence in lemmatized_data:
    for word in sentence:
        if sentence.count(word) < 5:
            sentence.remove(word)

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# словарь с указанием сколько раз встречается каждое слово
dictionary = Dictionary(res)
corpus = [dictionary.doc2bow(doc) for doc in res]


# словарь с указанием сколько раз встречается каждое слово
words_counter = Counter()
for sentence in lemmatized_data:
    for word in sentence:
        words_counter[word] += 1


C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
with open('word_frequency.csv','w', encoding='utf-8') as csvfile:
    columns = ['word', 'frequency']
    writer=csv.writer(csvfile)
    writer.writerow(columns)
    writer.writerows(words_counter.most_common())

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=10)
print(lda_model.show_topics())

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[(0, '0.050*"карантин" + 0.048*"я" + 0.035*"что" + 0.029*"в" + 0.026*"и" + 0.025*"не" + 0.017*"быть" + 0.017*"это" + 0.016*"за" + 0.014*"время"'), (1, '0.045*"не" + 0.028*"с" + 0.028*"в" + 0.027*"я" + 0.020*"на" + 0.020*"и" + 0.018*"карантин" + 0.012*"что" + 0.012*"это" + 0.011*"но"'), (2, '0.037*"в" + 0.029*"и" + 0.028*"не" + 0.022*"я" + 0.020*"карантин" + 0.017*"на" + 0.017*"ты" + 0.014*"быть" + 0.014*"с" + 0.014*"что"'), (3, '0.051*"и" + 0.037*"не" + 0.022*"в" + 0.021*"вирус" + 0.018*"что" + 0.017*"от" + 0.017*"а" + 0.017*"это" + 0.015*"они" + 0.010*"у"'), (4, '0.055*"я" + 0.019*"в" + 0.019*"не" + 0.019*"на" + 0.016*"по" + 0.015*"и" + 0.014*"а"

In [27]:
pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
